<a href="https://colab.research.google.com/github/mythogenesys/Andrew-NG-DL-practises/blob/main/ECG_Analysis_refactored_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#      REFACTORED: Full ECG Denoising Ablation & Validation Study
# ==============================================================================
# This notebook runs the complete ECG study using the new, refactored
# `run_training.py` and `run_validation.py` scripts.
# ==============================================================================
import os
from google.colab import drive

# ---
# [Step 1/4] SETUP: Mount Drive & Clone/Update Repository
# ---
print("--- [Step 1/4] Setting up the environment ---")
drive.mount('/content/drive')

GIT_REPO = "stpc-eeg"
GIT_PATH = "https://github.com/Mohan-CAS-and-hackathons/stpc-eeg.git"
PROJECT_DIR = f"/content/{GIT_REPO}"

if not os.path.exists(PROJECT_DIR):
    print(f"\nCloning repository...")
    !git clone -q {GIT_PATH} {PROJECT_DIR}
else:
    print(f"\nRepository exists. Pulling latest changes...")
    !cd {PROJECT_DIR} && git pull -q

os.chdir(PROJECT_DIR)
print(f"✅ Current directory: {os.getcwd()}")



--- [Step 1/4] Setting up the environment ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Cloning repository...
✅ Current directory: /content/stpc-eeg


In [ ]:
import os
os.chdir("/content")
print(os.getcwd())


/content


In [ ]:
!rm -rf stpc-eeg

In [ ]:
!pwd
!ls

/content/stpc-eeg
app.py		 convertGIFtoMP4.py  manuscript  README.md	   samples
assets		 docs		     models	 requirements.txt  src
CONTRIBUTING.md  LICENSE	     notebooks	 results	   tutorials


In [ ]:
# ---
# [Step 2/4] ENVIRONMENT: Install dependencies
# ---
print("\n--- [Step 2/4] Installing dependencies ---")
!pip install -q -r requirements.txt
# Ensure wfdb is up-to-date for PhysioNet downloads
# !pip install --upgrade -q wfdb




--- [Step 2/4] Installing dependencies ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<

In [ ]:
# ==============================================================================
#                      FINAL DIAGNOSTIC CELL
# ==============================================================================
import os
from google.colab import drive

# --- 1. SETUP: Mount Drive & Point to project ---
print("--- Setting up environment ---")
drive.mount('/content/drive')
PROJECT_DIR = "/content/stpc-eeg"
os.chdir(PROJECT_DIR)
print(f"Current directory: {os.getcwd()}")

# --- 2. DEFINE DATA PATH: This must be the correct path from your Drive ---
DATA_DIR_ECG = '/content/drive/MyDrive/stpc-eeg/data/mit-bih-arrhythmia-database-1.0.0'

# # --- 3. RUN DIAGNOSTIC: This will test wfdb access directly ---
# print("\n--- Running WFDB access check ---")
# !python3 src/experiments/check_wfdb_access.py --data_dir "{DATA_DIR_ECG}"

--- Setting up environment ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/stpc-eeg

--- Running WFDB access check ---
python3: can't open file '/content/stpc-eeg/src/experiments/check_wfdb_access.py': [Errno 2] No such file or directory


In [ ]:
print("\n--- [Step 3/4] Starting Ablation Study Training ---")

DRIVE_BASE_PATH = '/content/drive/MyDrive/stpc-eeg/'
DATA_DIR_ECG = os.path.join(DRIVE_BASE_PATH, 'data/mit-bih-arrhythmia-database-1.0.0')
DATA_DIR_NOISE = os.path.join(DRIVE_BASE_PATH, 'data/mit-bih-noise-stress-test-database-1.0.0')
MODEL_DIR = os.path.join(DRIVE_BASE_PATH, 'models')
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH_L1_ONLY = os.path.join(MODEL_DIR, 'denoiser_l1_only.pth')
MODEL_PATH_L1_GRAD = os.path.join(MODEL_DIR, 'denoiser_l1_grad.pth')
MODEL_PATH_STPC_FULL = os.path.join(MODEL_DIR, 'denoiser_stpc_full.pth')
MODEL_PATH_CLASSIFIER = os.path.join(MODEL_DIR, 'ecg_classifier_model.pth')

print("\n--- Training the ECG Classifier ---")
!python3 src/experiments/run_training.py ecg_classifier \
    --data_dir "/content/drive/MyDrive/stpc-eeg/data/mit-bih-arrhythmia-database-1.0.0" \
    --save_path "{MODEL_PATH_CLASSIFIER}" \
    --epochs 5

print("\n--- [Run 1/3] Training Denoiser with L1 Loss Only ---")
!python3 src/experiments/run_training.py ecg_denoiser \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --save_path "{MODEL_PATH_L1_ONLY}" \
    --epochs 5 --no-gradient-loss --no-fft-loss

print("\n--- [Run 2/3] Training Denoiser with L1 + Gradient Loss ---")
!python3 src/experiments/run_training.py ecg_denoiser \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --save_path "{MODEL_PATH_L1_GRAD}" \
    --epochs 5 --no-fft-loss

print("\n--- [Run 3/3] Training Denoiser with Full STPC Loss ---")
!python3 src/experiments/run_training.py ecg_denoiser \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --save_path "{MODEL_PATH_STPC_FULL}" \
    --epochs 5

print("\n✅ Ablation study training complete!")


--- [Step 3/4] Starting Ablation Study Training ---

--- Training the ECG Classifier ---
--- Training ECG Classifier ---
Extracting all annotated heartbeats from the dataset...
100% 48/48 [00:10<00:00,  4.55it/s]
Extracted a total of 108618 beats.
Label distribution: Counter({0: 90608, 2: 7235, 4: 7192, 1: 2781, 3: 802})
Epoch 1/5: 100% 679/679 [00:07<00:00, 85.72it/s, loss=0.0971] 
Validation Accuracy: 95.24%
Epoch 2/5: 100% 679/679 [00:06<00:00, 112.06it/s, loss=0.172]
Validation Accuracy: 97.04%
Epoch 3/5: 100% 679/679 [00:06<00:00, 112.84it/s, loss=0.122]
Validation Accuracy: 97.52%
Epoch 4/5: 100% 679/679 [00:03<00:00, 207.94it/s, loss=0.092]
Validation Accuracy: 97.67%
Epoch 5/5: 100% 679/679 [00:03<00:00, 210.98it/s, loss=0.0507]
Validation Accuracy: 98.21%
Classifier model saved to /content/drive/MyDrive/stpc-eeg/models/ecg_classifier_model.pth

--- [Run 1/3] Training Denoiser with L1 Loss Only ---
--- Training ECG Denoiser ---
Config: Gradient Loss=False, FFT Loss=False
Scann

In [ ]:
print("\n--- [Step 4/4] Starting End-to-End Validation for All Models ---")

RESULTS_DIR = os.path.join(DRIVE_BASE_PATH, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

OUTPUT_PREFIX_L1_ONLY = os.path.join(RESULTS_DIR, 'l1_only_final')
OUTPUT_PREFIX_L1_GRAD = os.path.join(RESULTS_DIR, 'l1_grad_final')
OUTPUT_PREFIX_STPC_FULL = os.path.join(RESULTS_DIR, 'stpc_full_final')

print("\n--- [Run 1/3] Validating L1 Only Model ---")
!python3 src/experiments/run_validation.py ecg_downstream \
    --denoiser_path "{MODEL_PATH_L1_ONLY}" \
    --classifier_path "{MODEL_PATH_CLASSIFIER}" \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --output_prefix "{OUTPUT_PREFIX_L1_ONLY}"

print("\n--- [Run 2/3] Validating L1 + Gradient Model ---")
!python3 src/experiments/run_validation.py ecg_downstream \
    --denoiser_path "{MODEL_PATH_L1_GRAD}" \
    --classifier_path "{MODEL_PATH_CLASSIFIER}" \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --output_prefix "{OUTPUT_PREFIX_L1_GRAD}"

print("\n--- [Run 3/3] Validating Full STPC Model ---")
!python3 src/experiments/run_validation.py ecg_downstream \
    --denoiser_path "{MODEL_PATH_STPC_FULL}" \
    --classifier_path "{MODEL_PATH_CLASSIFIER}" \
    --data_dir "{DATA_DIR_ECG}" \
    --noise_dir "{DATA_DIR_NOISE}" \
    --output_prefix "{OUTPUT_PREFIX_STPC_FULL}"

print("\n\n✅✅✅ Full ECG Study Complete! Check your Google Drive for results. ✅✅✅")


--- [Step 4/4] Starting End-to-End Validation for All Models ---

--- [Run 1/3] Validating L1 Only Model ---
2025-09-29 02:48:21.293433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759114101.313810   15683 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759114101.319911   15683 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759114101.335217   15683 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759114101.335242   15683 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same 